In [61]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import cycle

In [69]:
year_month = '202412'

In [71]:
# 解析年月字符串
year_month_dt = datetime.strptime(year_month, '%Y%m')

# 计算去年同期的年和月
previous_year = year_month_dt.year - 1
previous_month = year_month_dt.month

Mon = previous_month

# 格式化去年同期的年月
previous_year_month = f"{previous_year:04d}{previous_month:02d}"

# 计算去年同期月份的第一天
previous_month_first_day = year_month_dt.replace(year=previous_year, month=previous_month, day=1)

# 计算去年同期月份的最后一天
etd = (previous_month_first_day + timedelta(days=31)).replace(day=1) - timedelta(days=1)
etd = etd.strftime('%Y-%m-%d')

# 输出结果
print(f"Previous year month: {previous_year_month}")
print(f"ETD (End of the month of the previous year): {etd}")
print(Mon)

Previous year month: 202312
ETD (End of the month of the previous year): 2023-12-31
12


In [ ]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2024_v1.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))

In [ ]:
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))
vendor_nation_dict = dict(zip(vendor_mapping['Cleaned Vendor Name'],vendor_mapping['Nation']))
vendor_manager_dict = dict(zip(vendor_mapping['Cleaned Vendor Name'],vendor_mapping['Regional Manager']))

In [ ]:
blitz_vendor_info = pd.read_excel(r'C:\Medline\Vendor info\2024_BlitzVendor.xlsx')
blitz_ProjectLeader_dict = dict(zip(blitz_vendor_info['Vendor Name'],blitz_vendor_info['Project Leader']))
blitz_Category_dict = dict(zip(blitz_vendor_info['Vendor Name'],blitz_vendor_info['Category']))

In [ ]:
df_complaint_21_22 =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202401.xlsx',sheet_name=1)
df_complaint_21_22 = df_complaint_21_22.loc[df_complaint_21_22['Year'].isin([2021,2022])]

C:\Users\ypeng\AppData\Local\Temp\ipykernel_20060\2844925773.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_complaint_21_22 =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202401.xlsx',sheet_name=1)


In [ ]:
df_complaints_2024 = pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\2024\{} Complaint Data.xlsx'.format(year_month),sheet_name='2024 Complaint Database')
df_complaints_2024.drop(columns='New manufacturing complaints',inplace=True)

In [ ]:
df_complaints_2023 = pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\2023\{} Complaint Data.xlsx'.format(previous_year_month),sheet_name='2023 Complaint Database')

In [ ]:
df_complaints = pd.concat([df_complaints_2024,df_complaints_2023,df_complaint_21_22],ignore_index=True)

In [ ]:
# df_blitz = pd.concat([df_complaints_2024,df_complaints_2023],ignore_index=True)
# df_blitz.to_excel('blitz_complaints.xlsx',index=False)

In [ ]:
df_complaints_gy = (
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2021,2022,2023,2024])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter,
            Division = lambda d: d['Division'].map(str))
    .groupby(['Vendor Name','Division','Year','Quarter','Month'],as_index=False).size()
)
df_complaints_gy.rename(columns={'size':'Complaints','Vendor Name':'VendorName'},inplace=True)

In [ ]:
df_inspection_his = pd.read_excel(r'C:\Medline\3. QC trend report\Asia QC Trend Report 202001_202312.xlsx',sheet_name=0)

In [ ]:
df_inspection_us_2021_2022 =  df_inspection_his.loc[(df_inspection_his['Branch'] == 'US' ) & (df_inspection_his['Year'].isin([2021,2022])),['Vendors','Division','Year','Quarter','Month','#: Items inspected','#: Items reworked']]
df_inspection_us_2023 =  df_inspection_his.loc[(df_inspection_his['Branch'] == 'US' ) & (df_inspection_his['Year'].isin([2023])) & (df_inspection_his['Month']<= Mon),['Vendors','Division','Year','Quarter','Month','#: Items inspected','#: Items reworked']]

In [ ]:
df_inspection_us_current = pd.read_excel(r'C:\Medline\3. QC trend report\2024\Excel Report\Asia QC Trend Report US_202402.xlsx',sheet_name='Overall')
df_inspection_us_current =  df_inspection_us_current.loc[(df_inspection_us_current['Branch'] == 'US' ) & (df_inspection_us_current['Year'] == 2024),['Vendors','Division','Year','Quarter','Month','#: Items inspected','#: Items reworked']]

In [ ]:
df_inspection_us_current

,Vendors,Division,Year,Quarter,Month,#: Items inspected,#: Items reworked
0,APEX MEDICAL CORPORATION,75,2024.0,1.0,1.0,2,0
1,SUZHOU MECCAN IMP AND EXP CORP,14,2024.0,1.0,1.0,4,0
2,SUZHOU MECCAN IMP AND EXP CORP,15,2024.0,1.0,1.0,0,0
3,SUZHOU MECCAN IMP AND EXP CORP,18,2024.0,1.0,1.0,0,0
4,SUZHOU MECCAN IMP AND EXP CORP,20,2024.0,1.0,1.0,0,0
...,...,...,...,...,...,...,...
1638,SHANGHAI CHUANGSHI MEDICAL TECHNOLO,60,2024.0,1.0,1.0,6,0
1639,SHANGHAI CHUANGSHI MEDICAL TECHNOLO,60,2024.0,1.0,2.0,4,0
1640,XINGYU MEDICAL TECH CO LTD,40,2024.0,1.0,1.0,36,0
1641,XINGYU MEDICAL TECH CO LTD,40,2024.0,1.0,2.0,12,0


In [ ]:
df_inspection = pd.concat([df_inspection_us_current,df_inspection_us_2023,df_inspection_us_2021_2022],ignore_index=True)
df_inspection_us_groupby = df_inspection.groupby(['Vendors','Division','Year','Quarter','Month'],as_index=False).sum()

In [ ]:
df_inspection_us_groupby.rename(columns={'Vendors':'VendorName'},inplace=True)

In [ ]:
df_inspection_us_groupby['Year'] = df_inspection_us_groupby['Year'].map(int)
df_inspection_us_groupby['Quarter'] = df_inspection_us_groupby['Quarter'].map(int)
df_inspection_us_groupby['Month'] = df_inspection_us_groupby['Month'].map(int)

In [ ]:
df_BUOM_2021  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2021.xlsx', usecols='D,E,AB,AP,BI')
df_BUOM_2022  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2022.xlsx', usecols='D,E,AB,AP,BI')

In [ ]:
df_BUOM_2023  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US BUOM 202312.xlsx', usecols='C,D,H,P,Q')
df_BUOM_2023 = df_BUOM_2023.loc[df_BUOM_2023['ShipmentDt'] <= etd]

In [ ]:
df_BUOM_2024  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2024\US\US BUOM {}.xlsx'.format(year_month), usecols='D,E,P,X,Y')

In [ ]:
df_us_buom = pd.concat([df_BUOM_2024,df_BUOM_2023,df_BUOM_2022,df_BUOM_2021],ignore_index=True)

In [ ]:
df_us_buom.dropna(subset=['StrippedVendorNumber'],inplace=True)
df_us_buom['Year'] = df_us_buom['ShipmentDt'].dt.year
df_us_buom['Quarter'] = df_us_buom['ShipmentDt'].dt.quarter
df_us_buom['Month'] = df_us_buom['ShipmentDt'].dt.month

In [ ]:
df_us_buom.rename(columns={'StrippedVendorNumber':'VendorNumber'},inplace=True)
df_us_buom['VendorNumber'] = df_us_buom['VendorNumber'].map(int).map(str)

In [ ]:
df_us_buom_all = df_us_buom.copy()

In [ ]:
vendor_mapping_inspection = vendor_mapping.loc[(~vendor_mapping['Regional Manager'].isin(['Exemption','US vendor']))&(vendor_mapping['Regional Manager'].notnull()),'Vendor Number'].to_list()
df_us_buom_all = df_us_buom_all.loc[df_us_buom_all['VendorNumber'].isin(vendor_mapping_inspection)]
df_us_buom_all['VendorName'] = df_us_buom_all['VendorNumber'].apply(lambda x : vendor_mapping_dict.get(x,np.nan))
df_us_buom_all.rename(columns={'ProductDivisionCode':'Division'},inplace=True)
df_us_buom_all['Division'] = df_us_buom_all['Division'].map(int).map(str)

In [ ]:
df_us_buom_gy = df_us_buom_all.groupby(['VendorName','Division','Year','Quarter','Month'],as_index=False)['BUOMShipmentQty'].sum()

In [ ]:
df_us_buom_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16668 entries, 0 to 16667
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   VendorName       16668 non-null  object
 1   Division         16668 non-null  object
 2   Year             16668 non-null  int64 
 3   Quarter          16668 non-null  int64 
 4   Month            16668 non-null  int64 
 5   BUOMShipmentQty  16668 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 781.4+ KB


In [ ]:
df_complaints_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1761 entries, 0 to 1760
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   VendorName  1761 non-null   object
 1   Division    1761 non-null   object
 2   Year        1761 non-null   int64 
 3   Quarter     1761 non-null   int64 
 4   Month       1761 non-null   int64 
 5   Complaints  1761 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 82.7+ KB


In [ ]:
df_inspection_us_groupby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17651 entries, 0 to 17650
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   VendorName          17651 non-null  object
 1   Division            17651 non-null  object
 2   Year                17651 non-null  int64 
 3   Quarter             17651 non-null  int64 
 4   Month               17651 non-null  int64 
 5   #: Items inspected  17651 non-null  int64 
 6   #: Items reworked   17651 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 965.4+ KB


In [ ]:
raw_data = df_complaints_gy.merge(df_us_buom_gy,how ='outer',on=['VendorName','Division','Year','Quarter','Month']).merge(df_inspection_us_groupby,how = 'outer',on=['VendorName','Division','Year','Quarter','Month']).fillna(0)

In [ ]:
raw_data['Nation'] = raw_data['VendorName'].apply(lambda x : vendor_nation_dict.get(x,np.nan))
raw_data['Regional Manager'] = raw_data['VendorName'].apply(lambda x : vendor_manager_dict.get(x,np.nan))
raw_data['Project Leader'] = raw_data['VendorName'].apply(lambda x : blitz_ProjectLeader_dict.get(x,np.nan))
raw_data['Category'] = raw_data['VendorName'].apply(lambda x : blitz_Category_dict.get(x,np.nan))
raw_data['CPM'] = None

In [ ]:
raw_data.head()

,VendorName,Division,Year,Quarter,Month,Complaints,BUOMShipmentQty,#: Items inspected,#: Items reworked,Nation,Regional Manager,Project Leader,Category,CPM
0,A AND J HOLDING CO LTD,30,2021,1,1,18.0,0.0,0.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None
1,A AND J HOLDING CO LTD,30,2021,1,2,9.0,0.0,0.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None
2,A AND J HOLDING CO LTD,30,2021,1,3,16.0,0.0,0.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None
3,A AND J HOLDING CO LTD,30,2021,2,4,10.0,0.0,0.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None
4,A AND J HOLDING CO LTD,30,2021,2,5,4.0,0.0,0.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None


In [ ]:
raw_data = raw_data[['VendorName',
                     'Nation',
                     'Project Leader',
                     'Regional Manager',
                     'Category',
                     'Year',
                     'Month',
                     'Quarter',
                     'Division',
                     'Complaints',
                     'BUOMShipmentQty',
                     'CPM',
                     '#: Items inspected',
                     '#: Items reworked'
                     ]]

In [ ]:
raw_data = raw_data[~(raw_data['Complaints'] + raw_data['BUOMShipmentQty'] + raw_data['#: Items inspected'] + raw_data['#: Items reworked'] == 0)]

In [ ]:
raw_data.to_excel('us_raw_data.xlsx')